In [73]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [74]:
mt = pd.read_csv("metacritic_data.csv")

In [75]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [76]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
# tags = [('divmo','Maya Hawke', 'MOSS')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        new_tags.append((tag,uid))
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((tag,uid))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

✅  Noname - Room 25
✅  Rolo Tomassi - Time Will Die and Love Will Bury It
✅  Architects - Holy Hell
✅  Robyn - Honey
✅  Yves Tumor - Safe In The Hands Of Love
✅  Kacey Musgraves - Golden Hour
✅  Christine and the Queens - Chris
✅  Rival Consoles - Persona
✅  Jeff Rosenstock - POST-
✅  Idles - Joy As an Act of Resistance
✅  Behemoth - I Loved You at Your Darkest
✅  Marianne Faithfull - Negative Capability
✅  Janelle Monáe - Dirty Computer
✅  Car Seat Headrest - Twin Fantasy
✅  Hookworms - Microshift
✅  Alela Diane - Cusp
✅  Objekt - Cocoon Crush
✅  Daughters - You Won't Get What You Want
✅  Mitski - Be the Cowboy
✅  serpentwithfeet - Soil


In [77]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,20,20,20,20,20,20,20,16,20,20,1,20
unique,20,20,20,18,20,20,20,1,20,19,1,20
top,noname2018room25,Room 25,Noname,9 November 2018,3.59\n \n/ 5.00.5\n\n ...,"Jazz Rap, Conscious Hip Hop\nNeo-Soul, Jazz Po...","female vocals, introspective, conscious, po...",English,"[Self, Blaxploitation, Prayer Song, Window, Do...","[#7a7a7a, #cccccc]","French, English","#58 for 2018, #8,307 overall"
freq,1,1,1,2,1,1,1,16,1,2,1,1


In [78]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [79]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,20,20,20,20,20,20,20,17,20,20,20
unique,20,20,20,18,20,20,20,2,20,19,20
top,noname2018room25,Room 25,Noname,9 November 2018,3.59\n \n/ 5.00.5\n\n ...,"Jazz Rap, Conscious Hip Hop\nNeo-Soul, Jazz Po...","female vocals, introspective, conscious, po...",English,"[Self, Blaxploitation, Prayer Song, Window, Do...","[#7a7a7a, #cccccc]","#58 for 2018, #8,307 overall"
freq,1,1,1,2,1,1,1,16,1,2,1


In [80]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [81]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [82]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,noname2018room25,Room 25,Noname,14 September 2018,3.59,"Jazz Rap, Conscious Hip Hop\nNeo-Soul, Jazz Po...","female vocals, introspective, conscious, po...",English,"[Self, Blaxploitation, Prayer Song, Window, Do...","[#cccccc, #afaeaa, #c9c9c8, #9c9c9a, #b7b7b7, ...","#58 for 2018, #8,307 overall",7439
0,rolotomassi2018timyit,Time Will Die and Love Will Bury It,Rolo Tomassi,2 March 2018,3.53,"Post-Hardcore, Mathcore\nPost-Rock, Blackgaze,...","female vocals, anxious, ethereal, progressi...",English,"[Towards Dawn, Aftermath, Rituals, The Hollow ...","[#b5b4b4, #9b9ba3, #6a6979, #9d9d9f, #7c7b89, ...",#167 for 2018,2307
0,architects2018holell,Holy Hell,Architects,9 November 2018,3.47,"Metalcore\nDjent, Post-Hardcore","male vocals, death, heavy, passionate, ene...",English,"[Death Is Not Defeat, Hereafter, Mortal After ...","[#7a7a7a, #cccccc]",#240 for 2018,1408
0,robyn2018honney,Honey,Robyn,26 October 2018,3.44,"Dance-Pop, Electropop, House\nDeep House, Ambi...","female vocals, atmospheric, sensual, rhythm...",English,"[Missing U, Human Being, Because It's in the M...","[#da978f, #9a4739, #bb3623, #bba597, #7a5d3d, ...",#283 for 2018,3759
0,yvestumor2018safove,Safe in the Hands of Love,Yves Tumor,5 September 2018,3.61,"Neo-Psychedelia, Post-Industrial, Art Pop\nHyp...","psychedelic, eclectic, surreal, atmospheric...",English,"[Faith in Nothing Except in Salvation, Economy...","[#8d989c, #214d58, #6ea3d5, #29767c, #407c96]","#45 for 2018, #6,920 overall",9801
0,kaceymusgraves2018golour,Golden Hour,Kacey Musgraves,30 March 2018,3.51,"Country Pop, Singer-Songwriter\nContemporary C...","warm, mellow, bittersweet, love, female vo...",English,"[Slow Burn, Lonely Weekend, Butterflies, Oh, W...","[#169bc6, #5daac8, #fd8670, #f3b292]",#154 for 2018,6611
0,christineandthequeens2018chrris,Chris,Christine and the Queens,21 September 2018,3.36,"Synthpop\nElectropop, Alternative R&B, Synth F...","LGBT, playful, love, sexual, boastful, pa...","French, English","[French Version, Comme si on s'aimait, Damn, d...","[#a19690, #868d7d, #c1bd74, #b8c0b1, #bd9d8e]",#505 for 2018,1403
0,rivalconsoles2018perona,Persona,Rival Consoles,13 April 2018,3.42,"Microhouse, IDM\nAmbient Techno, Tech House, P...","atmospheric, instrumental, rhythmic, cold, ...",NaN,"[Unfolding, Persona, Memory Arc, Phantom Grip,...","[#a31403, #909694, #a8a8a8, #be361e]",#398 for 2018,1421
0,jeffrosenstock2018posost,POST-,Jeff Rosenstock,1 January 2018,3.36,"Power Pop, Indie Rock\nPop Punk","passionate, anthemic, melodic, introspectiv...",English,"[Mornin'!, USA, Yr Throat, All This Useless En...","[#869fa3, #95afb9, #8d9997, #4e5d62, #a9b6bc, ...",#485 for 2018,5156
0,idles2018joynce,Joy as an Act of Resistance.,Idles,31 August 2018,3.71,"Post-Punk, Post-Hardcore\nPunk Rock, Art Punk,...","political, aggressive, energetic, male voca...",English,"[Colossus, Never Fight a Man With a Perm, I'm ...","[#bb9b84, #524e4d, #8d754f, #a58554, #bfc2c4, ...","#20 for 2018, #2,156 overall",14242


In [83]:
xdf.to_csv("rym_add.csv")